# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

C:\Users\ASUS\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df=pd.read_csv("cal_housing_clean.csv")

In [3]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
medianHouseValue=df['medianHouseValue']

In [6]:
X=df.drop(['medianHouseValue'],axis=1)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test=train_test_split(X,medianHouseValue,test_size=0.30, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler= MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [11]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [12]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [13]:
housingMedianAge=tf.feature_column.numeric_column("housingMedianAge")
totalRooms=tf.feature_column.numeric_column("totalRooms")
totalBedrooms=tf.feature_column.numeric_column("totalBedrooms")
population=tf.feature_column.numeric_column("population")
households=tf.feature_column.numeric_column("households")
medianIncome=tf.feature_column.numeric_column("medianIncome")

In [14]:
feat_cols = [housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome]

In [15]:
feat_cols

[_NumericColumn(key='housingMedianAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalBedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='households', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='medianIncome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [16]:
input_func= tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,shuffle=True,num_epochs=1000)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [17]:
estimator=tf.estimator.DNNRegressor(feature_columns=feat_cols,hidden_units=[6,6,6])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ASUS\\AppData\\Local\\Temp\\tmpyswbogwx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002160E494DA0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [18]:
estimator.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ASUS\AppData\Local\Temp\tmpyswbogwx\model.ckpt.
INFO:tensorflow:loss = 484850340000.0, step = 0
INFO:tensorflow:global_step/sec: 221.131
INFO:tensorflow:loss = 425390870000.0, step = 100 (0.451 sec)
INFO:tensorflow:global_step/sec: 238.139
INFO:tensorflow:loss = 690525240000.0, step = 200 (0.421 sec)
INFO:tensorflow:global_step/sec: 237.965
INFO:tensorflow:loss = 545097700000.0, step = 300 (0.420 sec)
INFO:tensorflow:global_step/sec: 230.069
INFO:tensorflow:loss = 223319230000.0, step = 400 (0.437 sec)
INFO:tensorflow:global_step/sec: 233.945
INFO:tensorflow:loss = 681883000000.0, step = 500 (0.427 sec)
INFO:tensorflow:global_step/sec: 230.559
INFO:tensorflow:loss = 528486900000.0, step = 600 (0.432

INFO:tensorflow:global_step/sec: 225.886
INFO:tensorflow:loss = 139341960000.0, step = 7700 (0.445 sec)
INFO:tensorflow:global_step/sec: 231.854
INFO:tensorflow:loss = 115133140000.0, step = 7800 (0.429 sec)
INFO:tensorflow:global_step/sec: 231.896
INFO:tensorflow:loss = 158859920000.0, step = 7900 (0.431 sec)
INFO:tensorflow:global_step/sec: 230.401
INFO:tensorflow:loss = 202682800000.0, step = 8000 (0.435 sec)
INFO:tensorflow:global_step/sec: 232.052
INFO:tensorflow:loss = 90611020000.0, step = 8100 (0.434 sec)
INFO:tensorflow:global_step/sec: 228.532
INFO:tensorflow:loss = 86295085000.0, step = 8200 (0.435 sec)
INFO:tensorflow:global_step/sec: 232.136
INFO:tensorflow:loss = 192152390000.0, step = 8300 (0.430 sec)
INFO:tensorflow:global_step/sec: 227.155
INFO:tensorflow:loss = 160116410000.0, step = 8400 (0.441 sec)
INFO:tensorflow:global_step/sec: 233.003
INFO:tensorflow:loss = 39973890000.0, step = 8500 (0.428 sec)
INFO:tensorflow:global_step/sec: 230.563
INFO:tensorflow:loss = 145

INFO:tensorflow:global_step/sec: 235.234
INFO:tensorflow:loss = 187632030000.0, step = 15600 (0.422 sec)
INFO:tensorflow:global_step/sec: 232.877
INFO:tensorflow:loss = 117543666000.0, step = 15700 (0.429 sec)
INFO:tensorflow:global_step/sec: 231.51
INFO:tensorflow:loss = 66190934000.0, step = 15800 (0.433 sec)
INFO:tensorflow:global_step/sec: 234.212
INFO:tensorflow:loss = 165292380000.0, step = 15900 (0.429 sec)
INFO:tensorflow:global_step/sec: 232.304
INFO:tensorflow:loss = 59562370000.0, step = 16000 (0.428 sec)
INFO:tensorflow:global_step/sec: 230.86
INFO:tensorflow:loss = 87263530000.0, step = 16100 (0.433 sec)
INFO:tensorflow:global_step/sec: 202.387
INFO:tensorflow:loss = 73101890000.0, step = 16200 (0.494 sec)
INFO:tensorflow:global_step/sec: 233.166
INFO:tensorflow:loss = 135260200000.0, step = 16300 (0.429 sec)
INFO:tensorflow:global_step/sec: 231.1
INFO:tensorflow:loss = 64585400000.0, step = 16400 (0.433 sec)
INFO:tensorflow:global_step/sec: 227.616
INFO:tensorflow:loss = 

INFO:tensorflow:global_step/sec: 239.176
INFO:tensorflow:loss = 77177880000.0, step = 23500 (0.415 sec)
INFO:tensorflow:global_step/sec: 233.103
INFO:tensorflow:loss = 53856840000.0, step = 23600 (0.429 sec)
INFO:tensorflow:global_step/sec: 227.287
INFO:tensorflow:loss = 152123360000.0, step = 23700 (0.441 sec)
INFO:tensorflow:global_step/sec: 232.221
INFO:tensorflow:loss = 112051160000.0, step = 23800 (0.431 sec)
INFO:tensorflow:global_step/sec: 228.607
INFO:tensorflow:loss = 78086820000.0, step = 23900 (0.437 sec)
INFO:tensorflow:global_step/sec: 232.27
INFO:tensorflow:loss = 125480170000.0, step = 24000 (0.430 sec)
INFO:tensorflow:global_step/sec: 231.068
INFO:tensorflow:loss = 78579690000.0, step = 24100 (0.436 sec)
INFO:tensorflow:global_step/sec: 235.949
INFO:tensorflow:loss = 156616260000.0, step = 24200 (0.421 sec)
INFO:tensorflow:global_step/sec: 230.063
INFO:tensorflow:loss = 77872870000.0, step = 24300 (0.435 sec)
INFO:tensorflow:global_step/sec: 236.407
INFO:tensorflow:loss

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [22]:
pred_input_function= tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,shuffle=False,num_epochs=1)

In [23]:
prediction=estimator.predict(pred_input_function)

In [24]:
prediction=list(prediction)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ASUS\AppData\Local\Temp\tmpyswbogwx\model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [25]:
prediction

[{'predictions': array([236368.67], dtype=float32)},
 {'predictions': array([304292.4], dtype=float32)},
 {'predictions': array([217075.98], dtype=float32)},
 {'predictions': array([185964.28], dtype=float32)},
 {'predictions': array([273186.66], dtype=float32)},
 {'predictions': array([200292.86], dtype=float32)},
 {'predictions': array([228346.89], dtype=float32)},
 {'predictions': array([207511.38], dtype=float32)},
 {'predictions': array([217545.72], dtype=float32)},
 {'predictions': array([189716.89], dtype=float32)},
 {'predictions': array([206570.], dtype=float32)},
 {'predictions': array([225229.92], dtype=float32)},
 {'predictions': array([193911.92], dtype=float32)},
 {'predictions': array([179104.48], dtype=float32)},
 {'predictions': array([262027.53], dtype=float32)},
 {'predictions': array([177900.56], dtype=float32)},
 {'predictions': array([202913.97], dtype=float32)},
 {'predictions': array([187969.9], dtype=float32)},
 {'predictions': array([181544.78], dtype=float32)

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

# Great Job!